In [ ]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6"  color="red">ch14. 웹 데이터 수집 </font></b>

# 1절. BeautifulSoup과  parser
`pip in stall bs4` 아나콘다를 설치하면 자동 설치되는 패키지 7500개에 포함.

- 공식 Document site : https://beautiful-soup-4.readthedocs.io/en/latest/ 



In [ ]:
# 내 Local PC의 파일을 url()http://~처럼 접근
import requests                        # http 요청 처리하는 lib
from requests_file import FileAdapter  # file://프로토콜을 다루기 위함

In [ ]:
s = requests.Session() # http를 요청처리하는 세션 객체
s.mount("file://",FileAdapter())
# file:// 경로로 들어오면  이 요청은  c:(로컬파일)을 접근
response = s.get('file:///ai/source/01_python/data/ch14_sample.html')
response

In [ ]:
if response:
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다.')

In [ ]:
response.status_code
    # 200 : 정상
    # 400 : 없는페이지
    # 406 : get,post 오류

In [ ]:
response.content # 바이너리 형식의 html 내용

In [ ]:
response.content.decode('utf-8')

In [ ]:
print(response.content.decode('utf-8'))

In [ ]:
response.text  # HTML 파일의 텍스트 내용

In [ ]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, #response.text
                    "html.parser")
# soup

In [ ]:
# soup.select_one('선택자') : 해당선택자 처음 하나만 
el = soup.select_one('h1')  # 처음 나오는 h1태그 하나만
print('el :' , el)
print('el.text : ' , el.text)
print('el.string : ' , el.string)
print('el의 속성들 : ' , el.attrs)
print('el의 id속성 :', el.attrs['id'])  # el.attrs은 딕셔너리
print('el의 id속성 :', el.attrs.get('id'))
#print('el의 href속성 :', el.attrs.get['id]')
print('el의 href속성 :', el.attrs.get('href'))
print('el의 태그이름 :',el.name)

In [ ]:
# soup.select('선택자')     : 해당선택자 모두
els = soup.select('h1')  # h1태그를 list
print('els :',els)
print('els의 text들 :',[el.text for el in els] )                #els[0].text,els[1].text
print('els의 string들 :',[el.string for el in els] )
print('els의 속성들 :',[el.attrs for el in els])
print('els의 class 속성들  :', [el.attrs.get('class') for el in els])

In [ ]:
# soup.find(태그,속성)      : soup.select_one('선택자')와 유사
print('select_one :', soup.select_one('h1.css') )
print('find :', soup.find('h1', {'class':'css'}) ) # soup.select_one('h1.css')
print('find :', soup.find('h1', class_='css') )
print()
print('select_one :', soup.select_one('h1#text') )
print('find :', soup.find('h1', {'id':'text'}))

In [ ]:
# soup.find_all(태그,속성)  : soup.select('선택자')와 유사
print('select :',soup.select('h1.cc'))
print('find_all :',soup.find_all('h1',class_='css'))
print('find_all :',soup.find_all('h1',{'h1':'css'}))

In [ ]:
# 다른 경우
print('select :', soup.select('h1.css,span'))
print('find_all :', soup.find_all(['h1','span'],{'class':'css'}))
print('find_all :', soup.find_all(['h1','span'],class_='css') + soup.find_all('span'))

In [ ]:
# 없는 엘리먼트 찾기
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :',soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a',{'class','css'}))
print('find(None) :',soup.find('a',{'class':'css'}))

# 2절. 정적 웹데이터 수집(정적 웹크롤링)
## 2.1 BeutifulSoup을 활용한 html웹데이터수집
### 1)환율정보 가져오기(네이버증권 -> 시장지표
- https://finance.naver.com/marketindex/

In [ ]:
# 크롤링 허용범위는 사이트마다 ~/robots.txt에서 확인
    # Allow : / - 사이트의 모든 경로(/)에 대한 크롤링 허용

In [ ]:
# 방법1 
import requests 
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
# response.status_code
# response.text / response.content.decode('cp949')
soup = BeautifulSoup(response.text,"html.parser")

In [ ]:
# 방법2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response = urlopen(url)
response.status
#  response.read() / response.read().decode('cp949') / 
soup = BeautifulSoup(response,"html.parser")

In [ ]:
# div.head_info>Span.value (find함수)
prices = []
headinfos  =soup.find_all('div',class_ = 'head_info' )
for headinfo in headinfos:
    price = headinfo.find('span',class_='value')
    #print(price.text.replace(',',''))
    #print(float(''.join(price.text.split(','))))
    prices.append(float(''.join(price.text.split(','))))
print(prices)

In [ ]:
# div.head_info>Span.value (select함수)
prices = soup.select('div.head_info>span.value')
[float(p.text.replace(',','')) for p in prices]
#print(prices)

In [ ]:
# h_lst>span.blind
titles = soup.select('h3.h_lst>span.blind')
for t in titles:
    print(t.text , end='\t')
# print([t.text+'\t' for t in titles])

In [ ]:
# div.head_info>span>span.blind
units = soup.select('div.head_info>span>span.blind')
units = [unit.text for unit in units]
units.insert(7,'') # 7번째에 추가
units

In [ ]:
# div.head_info>span.blind
statuses = soup.select('div.head_info > span.blind')
for idx in range(len(statuses)):
    print(statuses[idx].text, end='\t')

In [ ]:
len(titles), len(prices), len(units), len(statuses)

In [ ]:
for idx in range(len(titles)):
    print("{} : {}{} - {}".format(titles[idx].text,
                                 prices[idx].text,
                                 units[idx],
                                 statuses[idx].text))

In [ ]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print("{} : {}{} -{}".format(title.text,
                                price.text,
                                unit,
                                status.text))

### 2) 이번주 로또 번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin  (google 

In [ ]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
#soup

In [ ]:
# 방법2
from urllib.request import urlopen
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = urlopen(url)
response.status
soup = BeautifulSoup(response, "html.parser")
# soup

In [ ]:
times = soup.select_one('div.win_result strong').text
date  = soup.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = soup.select('div.num.win  span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [ ]:
print(times, date)
print('당첨번호 ',lotto_number)
print('보 너 스 ', bonus_number)

### 3)다음 검색 리스트
```

no  title                href
0   한풀 꺾인 비트코인     https://v.daum.net/v/20251110094711892

```

In [ ]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
#     print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                           'title':item.text,
                           'link':item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)

In [ ]:
#  방법2 : 불가
from urllib.request import urlopen
from urllib.parse import quote
word =  quote('비트코인')  # url에 한글이 포한됨 경우 한글을 quote()로 변환
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = urlopen(url)
soup = BeautifulSoup(response,"html.parse")

items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
#     print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                           'title':item.text,
                           'link':item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)

In [ ]:
# 방법1 이어서.. 딕셔너리 대신 리스트로
items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
#     print(idx, item.text, item.attrs.get('href'))
    items_find_list.append([idx,item.text,item.attrs['href']])
pd.DataFrame(items_find_list,columns=['번호','제목','링크'])

In [ ]:
# find로 하기
items_find_list = []
# 'div.item-title > strong.tit-g > a'
item_titles=soup.find_all('div',class_='item-title')
for idx , item in enumerate(item_titles):
    a = item.find('a')
#     print(idx,item.text,a.attrs['href'])
#     items_find_list.append([idx,a.text,a.attrs['href']])
    items_find_list.append({
        'no':idx,
        'title':a.text,
        'link':a.attrs['href'],
    })
pd.DataFrame(items_find_list)

In [ ]:
# 다음 뉴스 검색 ( 원하는 키워드를 원하는 페이지를 가져오기 )
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword,page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력  -> list를 return'
#     https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}
#     response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword,'p':page}
    response = requests.get(url,params=params)
    soup = BeautifulSoup(response.text,'html.parser')
    items_find_list = []
    item_titles = soup.select('div.item-title > strong.tit-g >a')
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx,item.text,item.attrs['href']])
    return items_find_list

In [ ]:
collect_news_list('동대문',1)

In [ ]:
import time
keywords = ['청바지','동대문']
result0 = [] # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = [] # 동대문 1~3페이지까지 검색한 결과를 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번째 검색어 {keyword} 검색 결과 수집 중입니다 ===')
    for page in range(1,pages+1):
#         print(page,keyword)
        if i==0:
            result0.extend(collect_news_list(keyword,page))  # append는 결과가 3차원이 됨.=> 데이터프레임이 안됨   따라서, extend로 해야함
        elif i==1:
            result1.extend(collect_news_list(keyword,page))
        time.sleep(0.5)

In [ ]:
result0_df = pd.DataFrame(result0,columns=['no','title','link'])
result1_df = pd.DataFrame(result1,columns=['no','title','link'])
result0_df.head()

In [ ]:
result0_df.to_csv(f'data/ch14_{keywords[0]}.csv',index=False,encoding='cp949')
result1_df.to_csv(f'data/ch14_{keywords[1]}.csv',index=False)

### 4) User-Agent를 추가하여 크롤링
- 자동으로 브라우저를 통해 요청하는 것처럼 보이기

In [ ]:
# 방법2 : 
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인')  #  한글을 encoding 처리
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q='+word
print(url)
#headers ={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
# request =Request(url,headers=headers)
request =Request(url)
request.add_header('User-Agent',
                  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36')
response = urlopen(request)
soup = BeautifulSoup(response,'html.parser')
#soup

### * User-Agent로만 가능한 사이트 :방법2
```
- https://www.melon.com/chart/index.htm
    :  https://www.melon.com/robots.txt에서 User-Agent에서 봇이 지정
```


In [ ]:
# 방법1 
import  requests
url = 'https://www.melon.com/chart/index.htm'
melonpage = requests.get(url)
melonpage
# soup =BeautifulSoup(melonpage.text,'html.parser')
# soup


In [ ]:
# 방법 2 : User-Agent 추가
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/index.htm'
# melonpage = urlopen(url) # 에러남
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url,headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage,'html.parser')
soup

In [ ]:
# 방법1 : User-Agent 추가
import  requests
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage = requests.get(url,headers=headers)
soup = BeautifulSoup(melonpage.text, #melonpage.
                    'html.parser')
# soup
# div.wrap.t_center > span.rank
ranks = soup.select('td div.wrap.t_center > span.rank')
titles =soup.select('div.ellipsis.rank01 > span') # title.text.strip()
# [title.text.strip() for title in titles]
singers =soup.select('div.ellipsis.rank02') #[singer.text.strip()[:20]
# [singer.text.strip()[:20] for singer in singers]
# 1위 | 노래 제목 - 가수명
for rank,title,singer in zip(ranks,titles,singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

### 5)네이버 지식인으로 검색(open API 사용 X)
-특정 keyword를 특정페이지수만큼

In [ ]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '쳇지피티'
# url =f' https://kin.naver.com/search/list.nave'
# params = {'query':keyword}
# response = get(url,params=params)

url =f' https://kin.naver.com/search/list.naver?query={keyword}'
print(url)
response = get(url)
print(response.status_code)


soup = BeautifulSoup(response.text,'html.parser')
#soup


In [ ]:
# 방법2 
from urllib.request import  urlopen
from urllib.parse import quote  # keyword에 한글이 있을 경우
keyword = quote('쳇지피티')
url = f' https://kin.naver.com/search/list.naver?query={keyword}'
print(url)
response =urlopen(url)
print(response.status)
soup = BeautifulSoup(response,'html.parser')
#soup

In [ ]:
# 페이징 포함(1~3 페이지까지 가져오기)
from urllib.request import  urlopen
from urllib.parse import quote  # keyword에 한글이 있을 경우
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('쳇지피티')
page= 3
items_list = [] # 크롤링한 데이터를 담을 list(title,link)
for page in range(1,page+1):
    # print(page)
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    #url = f'https://kin.naver.com/search/list.naver?query=%EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0&page=3'
    #print(url)
    response = urlopen(url)
    #print(response.status) 
    soup = BeautifulSoup(response,'html.parser')
    items = soup.select('dt > a')
    #print(len(items))
     #print(items)
    for item in items:
        title = item.text
        link = item.attrs.get('href')
        #print(title,link)
        items_list.append({
            'title':title,
            'link':link
        })
#items_list  가독성떨어짐
df = pd.DataFrame(items_list)
print(df.shape)
print(df.loc[29,'link'])
df.sample()


## 2.2 openAPI사용 : json웹데이터 수집
### 1) 네이버 지식인으로 검색 (open API 사용 O)
-네이버 개발자샌터에서 애플리캐이션등록(이름/검색/WEB설정 http://localhost)
- .env파일에 CLIENT_ID/CLIENT_SECRET 환경변수 저장
- 환경변수를 읽기 위해서  `pip install dotenv`
- 특정 keyword를 지식검색(데이터수 30개)

In [ ]:
%pip install dotenv

In [ ]:
#  환경변수 읽어오기
from dotenv import load_dotenv
import os
load_dotenv()   # 현 소스와 같은 폴더 내의 .env를 메모리에 load
                # 다른 경우 load_dotenv(dorenv_path='.env')
# print(os.getenv('CLIENT_ID'))
# print(os.getenv('CLIENT_SECRET'))

In [ ]:
# 방법2
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/kin?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/kin.xml?query=" + encText # XML 결과
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)

headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
request = urllib.request.Request(url,headers=headers)

response = urllib.request.urlopen(request)

rescode = response.status
if(rescode==200):
    response_body = response.read()       #json파일
    #print(type(response_body.decode('utf-8')))
    #response_body의 문자내용을 json스타일의 딕셔너리로 변환    print(json.loads(response_body.decode('utf-8')))
    print(response_body.decode('utf-8')[:50])  # response_body.decode('utf-8') :json스타일의 문자
else:
    print("Error Code:" + rescode)
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link  = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(items_list, columns=['title', 'link', 'description']).head()

In [ ]:
# 방법1
import requests
import pandas as pd
import json # response 택스트를 json스타일의 딕셔너리로 
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '쳇지피티'
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f"https://openapi.naver.com/v1/search/kin?query={query}&display=30"
# response = requests.get(url,headers=headers)
url = "https://openapi.naver.com/v1/search/kin"
params = {'query':query,'display':30}
response = requests.get(url,params=params,headers=headers)
# print(response.text[:500])
# items = json.loads(response.text)['items']
items = response.json()['items']
items_list = []
for item in items:
    title =item['title'].replace('<b>','').replace('</b>','')
    link = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title,link,description])
pd.DataFrame(items_list,columns=['title','link','description']).head()

### quiz)네이버 openAPI를 이용해서 청바지 이미지 100건의 데이터를 ch14_청바지.csv출력
```
    제목, 링크, 썸네일,sizeheight,sizewidth
```

In [ ]:
# 방법1
import os
import requests
import pandas as pd
# import json # requests.json()을 사용하므로, 명시적으로 json 모듈을 사용할 필요는 줄어듭니다.
# import urllib.parse # requests를 사용하므로 이 모듈도 필요 없습니다.

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

query = "청바지" # 검색어

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret
}

# requests.get()의 params 인자에 쿼리와 display를 넘기면 requests가 자동으로 처리해줍니다.
# query 인코딩을 직접 하지 않아도 되어서 더 깔끔해요.
params = {
    'query': query,
    'display': 100 # 한 번에 가져올 이미지 수
}

url = "https://openapi.naver.com/v1/search/image"

response = requests.get(url, headers=headers, params=params)

print(response) 

# 응답 상태 코드가 200 (성공)인지 확인
if response.status_code == 200:
    result_json = response.json()
    items = result_json.get('items', [])
    
    # print(response.text[:500]) 

    items_list = []
    for item in items:
        title = item.get('title', '').replace('<b>','').replace('</b>','')
        link = item.get('link', '')         # 원본 이미지 URL
        thumbnail = item.get('thumbnail', '') # 썸네일 이미지 URL
        sizeheight = item.get('sizeheight', '') # 이미지 높이
        sizewidth = item.get('sizewidth', '')   # 이미지 너비
        
        items_list.append([title, link, thumbnail, sizeheight, sizewidth])
    
    # 추출한 데이터를 Pandas DataFrame으로 생성합니다.
    df = pd.DataFrame(items_list, columns=['제목', '링크', '썸네일', '높이', '너비'])
    print(df.head())
    
else:
    # 요청 실패 시 오류 코드와 응답 내용을 출력합니다.
    print(f"Error Code: {response.status_code}")
    print(response.text)

In [ ]:
# 선생님것. 방법2
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법2)'
    from urllib.request import urlopen, Request
    from urllib.parse import quote
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    encText = quote(query)
    url = f'https://openapi.naver.com/v1/search/image?query={encText}&display=100'
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    request = Request(url, headers=headers)
    response = urlopen(request)
    # print(response.read().decode('utf-8'))
    items = json.loads(response.read().decode('utf-8'))['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

In [ ]:
# 방법1(선생님)
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

In [ ]:
print(df.loc[0, '링크'])
print(df.loc[0, '썸네일'])

In [ ]:
def save_image(attr, idx, link, query):
    'link의 이미지를 image/attr_idx_query.확장자로 local에 저장'
    import requests
    file_extension = link.split('.')[-1] #확장자
    quote_index = file_extension.find('?')   # 확장자 뒤에 ?가 있는 위치    .jpg?w=780
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
    img = requests.get(link).content  # 바이너리
    with open(f'image/{attr}_{idx+1}_{query}.{file_extension}','wb') as f:
        f.write(img)
save_image('메인',0,df.loc[0, '링크'],'청바지')

In [ ]:
# 합치기
# 방법1(선생님)
def get_image_list_save_image(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = [] # 정보가 담길 list
    for idx, item in enumerate(items):
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        #이미지 파일저장
        save_image('메인',idx,link,query)
        save_image('썸네일',idx,link,query)
        if (idx%20 == 0)& (idx !=0):
            print(f'====={idx}% 진행 중 =====')
    print('+++++++ 완 료 ++++++++')
    return pd.DataFrame(items_list)
df = get_image_list_save_image("청바지")
df.to_csv('image/ch14_청바지.csv')
df.sample()

### 2.3 XML 웹데이터 수집
- RSS / open API 을 통한 XML 웹데이터 수집
### 1) 전국 날씨 RSS를  BeautifulSoup을 이용한 크롤링
- 구글에 "기상청 RSS" 검색한 첫번째 사이트 이동

In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
items_list = []
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml'
# 방법1
# target = requests.get(url)
# soup = BeautifulSoup(target.text, "xml") # pip install lxml 필요없음
# 방법2
target = urlopen(url)
soup = BeautifulSoup(target, "xml")
local_tas = soup.select('local_ta')
for local_ta in local_tas:
    local_name = local_ta.select_one('local_ta_name').text
    week1_normalYear = local_ta.select_one('week1_local_ta_normalYear').text
    week1_similarRange = local_ta.select_one('week1_local_ta_similarRange').text
    week1_minVal = local_ta.select_one('week1_local_ta_minVal').text
    week1_similarVal = local_ta.select_one('week1_local_ta_similarVal').text
    week1_maxVal = local_ta.select_one('week1_local_ta_maxVal').text
    items_list.append({
        '지역':local_name,
        '1주평년기온':week1_normalYear,
        '1주기온범위':week1_similarRange,
        '1주낮을확률':week1_minVal,
        '1주비슷할확률':week1_similarVal,
        '1주높을확률':week1_maxVal
    })
pd.DataFrame(items_list)

### 2) XML응답하는  open API
- data.go.kr에서 
    - 서울특별시_노선정보조회 서비스(버스ID,정류장목록과정류장ID)
    - 서울특별시_버스위치정보조회 서비스(실시간 버스 위치 목록)

### ☆STEP1 버스번호의 버스ID박아오기
- 서울특별시_노선정보조회 서비스 -3번 가능(getBusRouteList) 이용

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('KEY'))
# print(os.getenv('KEY1'))

True

In [13]:
# 내 pc에 저장하기
from bs4 import BeautifulSoup
# urlretrieve(url, 저장경로) : url의 파일을 저장경로에 저장
from urllib.request import urlretrieve
from urllib.parse import quote
# busNum = quote('구로09')
busNum = '162'
key = os.getenv('KEY')
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
print(url)
savefilename1 = 'data/ch14_busInfo.xml'
urlretrieve(url,savefilename1)  # xml(url)파일을 savefilename1d으로 저장
with open(savefilename1,encoding='utf-8') as f:
    xml = f.read()
soup  = BeautifulSoup(xml,'xml')

http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey=17fe52e6614af862f58a7555493dcf98e50eb55731d19ac363d85a41ce2de47a&strSrch=162


In [25]:
#  pc에 저장하지 않고 
import requests
from urllib.parse import quote
# busNum = quote('구로09')
busNum = '162'
key = os.getenv('KEY')
url1 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
# print(url1)
response = requests.get(url1)
soup = BeautifulSoup(response.text,'xml')
# soup

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
#     print(busRouteNm)
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        break;
print('busRouteId = ', busRouteId)

busRouteId =  100100034


### ☆STEP2.  버스id로 버스정류장목록받아오기(정류장명,id,경도,위도)
- 서울특별시_노선정보조회 서비스 -4번 기능(getStaionsByRouteList)

In [32]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
print(url2)
response = requests.get(url2)
soup = BeautifulSoup(response.content,'xml')
itemLists = soup.find_all('itemList')
print(f'{busNum}번 정류장 갯수 : {len(itemLists)}')
bus_station = []
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text  # 정류소이름
    station   = itemList.find('station').text    # 정류소 id
    gpsX      = itemList.find('gpsX').text       # 경도
    gpsY      = itemList.find('gpsY').text       # 위도
    bus_station.append([stationNm,station,gpsX,gpsY])
df_station = pd.DataFrame(bus_station,columns=['정류소명','id','경도','위도'])
df_station.head()
    

http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey=17fe52e6614af862f58a7555493dcf98e50eb55731d19ac363d85a41ce2de47a&busRouteId=100100034
162번 정류장 갯수 : 77


,정류소명,id,경도,위도
0,정릉산장아파트,107000071,127.003343,37.616712
1,정릉4동주민센터.경국사,107000073,127.006345,37.613529
2,북한산보국문역2번출구,107000518,127.0079858233,37.612293899
3,성북청수도서관.정릉4동성당,107000075,127.0084193769,37.6115696748
4,정릉시장입구,107000077,127.0098212542,37.6084653256


### ☆STEP3. 버스id로 실시간위치정보를 받아오기(차량번호,혼잡도,최종정류장id,다음정류장id,도착소요시간)
- 서울특별시_버스위치정보조회 서비스 - 2번(getBusPosByRtidList)

In [44]:
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey={key}&busRouteId={busRouteId}'
# print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text,'xml')
itemLists = soup.select('itemList')
print(f'운행중인 버스는 {len(itemList)}대입니다')
bus_position = []
for itemList in itemLists:
    plainNo = itemList.select_one('plainNo').text  # 차량번호
    congetion = itemList.select_one('congetion').text # 혼잡도(congetion)
     # 0 : 없음, 3 : 여유, 4 : 보통, 5 : 혼잡, 6 : 매우혼잡
    congetion ="없음" if congetion=='0'\
        else  '여유' if congetion =='3' \
        else  "보통" if congetion =='4' \
        else  '혼잡' if congetion =='5'  else '매우혼잡'
    bus_position.append({
        '차량번호':plainNo,
        '혼잡도':congetion
    })
df_position = pd.DataFrame(bus_station)
df_position
        

운행중인 버스는 23대입니다


""
